# Micro-benchmarks and timeit

In [1]:
import StringIO, cStringIO

def build_concat(strings):
    result = ''
    for s in strings:
        result += s
    return result

def build_join(strings):
    return ''.join(strings)

def build_sio(strings):
    sio = StringIO.StringIO()
    for s in strings:
        sio.write(s)
    return sio.getvalue()

def build_csio(strings):
    sio = cStringIO.StringIO()
    for s in strings:
        sio.write(s)
    return sio.getvalue()

strings = [str(i) for i in range(1000)]

In [2]:
build_concat(strings) == build_join(strings) == build_sio(strings) == build_csio(strings)

True

In [3]:
import timeit

In [9]:
for fname in ('concat', 'join', 'sio', 'csio'):
    elapsed = timeit.timeit(
        'build_%s(strings)' % fname, 
        In[1], number=10000)
    print(fname, elapsed)

('concat', 0.6744589805603027)
('join', 0.09292483329772949)
('sio', 7.163503885269165)
('csio', 2.6579480171203613)


## Jupyter Notebook (and IPython) provide a nice helper

We can use `%timeit` (or usually just `timeit`) to run a version of `timeit` in IPython

In [10]:
%timeit build_concat(strings)

10000 loops, best of 3: 69 µs per loop


In [11]:
%timeit build_join(strings)

100000 loops, best of 3: 9.35 µs per loop


In [12]:
%timeit build_sio(strings)

1000 loops, best of 3: 690 µs per loop


In [13]:
%timeit build_csio(strings)

1000 loops, best of 3: 287 µs per loop


## Results of our microbenchmark

- `''.join()` is the fastest, around 7x faster than concatenating strings
- `cStringIO` is around 2-3x faster than `StringIO`, is around 30x slower than `''.join`
- You should use `''.join` when you need to build a big string from a list of strings

# Profiling Python Code

You can profile an entire script by running it via `python -m cProfile` to get summary information about the whole thing:

In [17]:
!cat ../examples/Profiling/profiletest.py

import re
re.compile("foo|bar")


In [19]:
!python -m cProfile ../examples/Profiling/profiletest.py

         195 function calls (190 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 profiletest.py:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:228(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:256(_optimize_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:433(_compile_info)
        2    0.000    0.000    0.000    0.000 sre_compile.py:546(isstring)
        1    0.000    0.000    0.000    0.000 sre_compile.py:552(_code)
        1    0.000    0.000    0.000    0.000 sre_compile.py:567(compile)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:64(_compile)
        5    0.000    0.000    0.000    0.000 sre_parse.py:137(__len__)
       12   

In [20]:
!python -m cProfile -s time ../examples/Profiling/profiletest.py

         195 function calls (190 primitive calls) in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:64(_compile)
      3/1    0.000    0.000    0.000    0.000 sre_parse.py:151(getwidth)
        2    0.000    0.000    0.000    0.000 sre_parse.py:395(_parse)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:433(_compile_info)
        1    0.000    0.000    0.000    0.000 sre_compile.py:552(_code)
        1    0.000    0.000    0.000    0.000 sre_parse.py:317(_parse_sub)
       10    0.000    0.000    0.000    0.000 sre_parse.py:193(__next)
        1    0.000    0.000    0.000    0.000 sre_parse.py:706(parse)
        1    0.000    0.000    0.000    0.000 sre_compile.py:567(compile)
        1    0.000    0.000    0.000    0.000 profiletest.py:1(<module>)
        1    0.000    0.000    

### Direct profiling

We can also profile just a few Python statements or a function:

In [21]:
import re
import cProfile

In [22]:
cProfile.run('re.compile("foo|bar")')

         195 function calls (190 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:228(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:256(_optimize_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:433(_compile_info)
        2    0.000    0.000    0.000    0.000 sre_compile.py:546(isstring)
        1    0.000    0.000    0.000    0.000 sre_compile.py:552(_code)
        1    0.000    0.000    0.000    0.000 sre_compile.py:567(compile)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:64(_compile)
        5    0.000    0.000    0.000    0.000 sre_parse.py:137(__len__)
       12    0.000

We can also save the statistics to a file for analysis later:

In [23]:
cProfile.run('re.compile("foo|bar")', 're-stats')

For analysis, we use the `pstats` module:

In [24]:
import pstats

In [25]:
p = pstats.Stats('re-stats')

In [26]:
p.print_stats()

Sat Apr 15 16:03:07 2017    re-stats

         4 function calls in 0.000 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /Users/rick446/.virtualenvs/intermediate-python/lib/python2.7/re.py:192(compile)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 /Users/rick446/.virtualenvs/intermediate-python/lib/python2.7/re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)




<pstats.Stats instance at 0x105cfbcf8>

In [28]:
p.strip_dirs()
p.print_stats()

Sat Apr 15 16:03:07 2017    re-stats

         4 function calls in 0.000 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)




<pstats.Stats instance at 0x105cfbcf8>

In [29]:
p.sort_stats('time')

<pstats.Stats instance at 0x105cfbcf8>

In [30]:
p.print_stats()

Sat Apr 15 16:03:07 2017    re-stats

         4 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




<pstats.Stats instance at 0x105cfbcf8>

In [31]:
p.print_stats(10)

Sat Apr 15 16:03:07 2017    re-stats

         4 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 re.py:230(_compile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:192(compile)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




<pstats.Stats instance at 0x105cfbcf8>

IPython also has a magic function to help us here, as well:

In [35]:
prun re.compile('foo|bar')

# Instrumenting high-performance code

There are times when we want to profile, but we don't want to incur the performance penalty. For instance, we might want to see the profile of a running production system, without impacting its performance in a major way.

For that, we can profile a _sample_ of the calls to a function.

In [36]:
import random
import functools

def instrument(profiler, probability=0.10):
    '''Profile some of the calls to the decorated function.
    
    The default probability of profiling a call is 10%.
    '''
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if random.random() < probability:
                try:
                    profiler.enable()
                    return func(*args, **kwargs)
                finally:
                    profiler.disable()
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator

In [37]:
import cProfile
prof = cProfile.Profile()

In [38]:
@instrument(prof, 0.25)
def build_join(strings):
    return ''.join(strings)

In [39]:
for x in xrange(10000):
    build_join(strings)

In [40]:
prof.print_stats()

         7509 function calls in 0.024 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2503    0.001    0.000    0.024    0.000 <ipython-input-38-f3bd122e11df>:1(build_join)
     2503    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
     2503    0.023    0.000    0.023    0.000 {method 'join' of 'str' objects}




# Lab

Using timeit, compare the performance of searching text using a compiled versus an uncompiled regular expression:

In [41]:
text = '''The quick brown fox jumps over the lazy dog'''

In [42]:
import re
pattern = 'fox'
re_fox = re.compile(pattern)

Use cProfile and pstats to profile a function that uses an uncompiled `re.search` to search the text

Instrument the function with the above decorator, and use %timeit to compare the profiling overhead between profiling 1% of the time and 100% of the time